In [5]:
import os
import re
import math
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm
from tools.settings import *
from keras.preprocessing import image
from tools.train_val_test_spliter import split

In [112]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.4 Validation=0.1
Done


In [26]:
train = pd. read_csv(os.path.join(dataset_path, "train.csv"))
test = pd. read_csv(os.path.join(dataset_path, "test.csv"))
val = pd. read_csv(os.path.join(dataset_path, "val.csv"))

In [17]:
train.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch


In [18]:
test.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [19]:
val.head()

,Video_url,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,kick
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [27]:
print(train.shape)
print(test.shape)
print(val.shape)

(143, 2)
(111, 2)
(27, 2)


In [7]:
def convert_to_frame(data, folder_name):
    '''
    Generated filenames format dataset_path/folder_name/video_name_frame{number}_action.jpg
    '''
    directory = os.path.join(dataset_path, folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        action = data['action'][i]
        video_name_list = video_file.split('/')[-1].split('.')
        video_name_list = video_name_list[:-1]
        video_name = ""
        for n in video_name_list:
            video_name += n
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [89]:
convert_to_frame(train, train_frames_path_name)

100%|██████████| 143/143 [00:50<00:00,  2.84it/s]

Successfully Converted


In [120]:
def create_paths_csv(directory, file_name):
    images = os.listdir(directory)
    images_path_list = []
    images_action_list = [] 
    for image in images:
        images_path_list.append(directory + image)
        images_action_list.append(image.split('.')[0].split('_')[-1])
    df = pd.DataFrame()
    df['image'] = images_path_list
    df['action'] = images_action_list
    print(os.path.join(dataset_path, file_name+'.csv'))
    df.to_csv(os.path.join(dataset_path, file_name+'.csv'), index=False)

In [121]:
create_paths_csv(train_frames_path, train_frames_path_name)

/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-time-action-recognition-from-video-footage/dataset/train_frames.csv


In [2]:
train_image = pd.read_csv(os.path.join(dataset_path, 'train_frames.csv'))
train_image.head()

,image,action
0,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,punch
1,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
2,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
3,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap
4,/home/mushfiqul/Mushfiqul/CSE/Thesis2.0/real-t...,slap


In [3]:
print(train_image.shape)

(620, 2)


In [6]:
train_image_value = []
for i in tqdm(range(train_image.shape[0])):
    img = image.load_img(train_image['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img / 255
    train_image_value.append(img)

X = np.array(train_image_value)
X.shape

100%|██████████| 620/620 [00:07<00:00, 86.24it/s]


(620, 224, 224, 3)

     |████████████████████████████████| 2.2 MB 836 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
